In [1]:
import numpy as np
from CreateMaskForWindow import CreateMaskForWindow
from DistributeValue import DistributeValue

In [2]:
def PoolBackward(dA, cache, mode = None):
    """
    Implements the backward pass of the pooling layer
    
    Arguments:
    dA -- gradient of cost with respect to the output of the pooling layer, same shape as A
    cache -- cache output from the forward pass of the pooling layer, contains the layer's input and hparameters 
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    dA_prev -- gradient of cost with respect to the input of the pooling layer, same shape as A_prev
    """
    ### START CODE HERE ###
    
    # Retrieve information from cache (≈1 line)
    (Aprev, hyperparameters, modeCache) = cache
    
    mode = "max" if mode == None and modeCache == None else modeCache if mode == None else mode 
    
    # Retrieve hyperparameters from "hparameters" (≈2 lines)
    stride = hyperparameters["stride"]
    f = hyperparameters["f"]
    
    # Retrieve dimensions from A_prev's shape and dA's shape (≈2 lines)
    (m, nHprev, nWprev, nCprev) = Aprev.shape
    (m, nH, nW, nC) = dA.shape
    
    # Initialize dA_prev with zeros (≈1 line)
    dAprev = np.zeros(Aprev.shape)
    
    for m1 in range(m):                                               # loop over the training examples
        for i,h1 in enumerate(range(0,nHprev-f+1, stride)):           # loop on the vertical axis
            for j,w1 in enumerate(range(0,nWprev-f+1, stride)):       # loop on the horizontal axis
                for c1 in range(nC):                                  # loop over the channels (depth)
                    # Compute the backward propagation in both modes.

                    if mode == "max": 
                        dAprev[m1,h1:h1+f,w1:w1+f,c1] += CreateMaskForWindow(Aprev[m1,h1:h1+f,w1:w1+f,c1]) * dA[m1,i,j,c1]
                    elif mode == "average":
                        s = DistributeValue(dA[m1,i,j,c1], (f,f))
                        dAprev[m1,h1:h1+f,w1:w1+f,c1] += s

                  
    ### END CODE ###
    
    # Making sure your output shape is correct
    assert(dAprev.shape == Aprev.shape)
    
    return dAprev

In [3]:
np.random.seed(1)
Aprev = np.random.randn(5,5,3,2)
hyperparameters = {"stride": 2, "f":1}
dA = np.random.randn(5,4,2,2,)
cache2 = (Aprev, hyperparameters, "max")
modes = ("max","average")

for mode in modes:
    dAprev = PoolBackward(dA, cache2, mode)
    print("\nmode: ", mode)
    print("dAprev{}: \n{}".format(dAprev[0,:,:,0].shape,dAprev[0,:,:,0]))


mode:  max
dAprev(5, 3): 
[[-0.31011677  0.          1.0388246 ]
 [ 0.          0.          0.        ]
 [ 0.44136444  0.         -0.13644474]
 [ 0.          0.          0.        ]
 [ 0.01740941  0.         -0.51709446]]

mode:  average
dAprev(5, 3): 
[[-0.31011677  0.          1.0388246 ]
 [ 0.          0.          0.        ]
 [ 0.44136444  0.         -0.13644474]
 [ 0.          0.          0.        ]
 [ 0.01740941  0.         -0.51709446]]
